In [2]:
import time
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load data
params_df = pd.read_csv('mosfet_params_v9.csv')
measurements_df = pd.read_csv('measurements_v9.csv')

# --- Preprocess data (copied from your notebook/PDF) ---
X_list = []
mosfet_ids = []
grouped = measurements_df.groupby('MOSFET_ID')
for mosfet_id, group in grouped:
    group_sorted = group.sort_values('meas_index')
    if len(group_sorted) == 561:
        features = group_sorted[['VGS', 'VDS', 'ID']].values.flatten()
        X_list.append(features)
        mosfet_ids.append(mosfet_id)
X = np.array(X_list)

params_df.set_index('MOSFET_ID', inplace=True)
# Ensure y targets VTO specifically as in the PDF [cite: 2]
y_list = [params_df.loc[mid, ['VTO']].values for mid in mosfet_ids]
y = np.array(y_list).reshape(-1, 1)

# --- Split data ---
# IMPORTANT: Use the same random_state as during training for consistency [cite: 4]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Scale inputs ---
input_scaler = StandardScaler()
# Fit the input scaler ONLY on the training data
X_train_scaled = input_scaler.fit_transform(X_train)
# Apply the SAME scaler to the test data
X_test_scaled = input_scaler.transform(X_test)

# --- Scale outputs (Target: VTO) ---
output_scaler = StandardScaler()
# Fit the output scaler ONLY on the training data [cite: 4]
y_train_scaled = output_scaler.fit_transform(y_train)
# Note: We don't strictly need y_test_scaled unless evaluating loss in scaled terms,
# but we need the FITTED output_scaler later.

# --- Load model ---
model = load_model('mosfet_VTO_model_v9.keras') # [cite: 19]

# --- Adjust test set size (if necessary) ---
num_test_devices = X_test_scaled.shape[0]
print(f"Number of devices in original test set: {num_test_devices}")

# Select the first 10000 devices from the *original* test set splits
# Ensure we select corresponding X and y samples
if num_test_devices >= 10000:
    X_test_scaled_10000 = X_test_scaled[:10000]
    y_test_10000 = y_test[:10000] # Use the original y_test slice
    print(f"Using first 10000 devices from the test set")
else:
    # Handle cases with fewer than 10000 test devices if needed
    # For now, we'll just use the available ones. Adjust logic as needed.
    print(f"Warning: Test set has fewer than 10000 devices ({num_test_devices}). Using all available.")
    X_test_scaled_10000 = X_test_scaled
    y_test_10000 = y_test # Use the original y_test

# --- Measure prediction time ---
start_time = time.time()
# Predict using the potentially adjusted test set
predictions_scaled = model.predict(X_test_scaled_10000)
end_time = time.time()

# --- Inverse transform predictions ---
# Use the output_scaler that was fitted on y_train [cite: 17]
predictions_unscaled = output_scaler.inverse_transform(predictions_scaled)

# --- Calculate time taken ---
time_taken = end_time - start_time
num_predictions = predictions_unscaled.shape[0]
predictions_per_second = num_predictions / time_taken

# --- Print timing results ---
print(f"\nTime taken to predict {num_predictions} devices: {time_taken:.3f} seconds")
print(f"Average predictions per second: {predictions_per_second:.2f}")
print(f"Average time per prediction: {(time_taken / num_predictions) * 1000:.3f} milliseconds")

# --- Calculate Percentage Error Statistics (using unscaled values) ---
# Ensure y_test_10000 corresponds to the predictions_unscaled
# Avoid division by zero if any y_test values are 0
abs_percentage_errors = np.abs((predictions_unscaled - y_test_10000) / y_test_10000[y_test_10000 != 0]) * 100

print("\nPrediction Error Statistics (Corrected):")
print(f"Mean Absolute Percentage Error (MAPE): {np.mean(abs_percentage_errors):.2f}%")
print(f"Median Absolute Percentage Error: {np.median(abs_percentage_errors):.2f}%")
print(f"Minimum Percentage Error: {np.min(abs_percentage_errors):.2f}%")
print(f"Maximum Percentage Error: {np.max(abs_percentage_errors):.2f}%")

# Optional: Calculate other metrics like R2, MSE, RMSE as in the PDF [cite: 19]
from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(y_test_10000, predictions_unscaled)
mse = mean_squared_error(y_test_10000, predictions_unscaled)
rmse = np.sqrt(mse) # Or use root_mean_squared_error

print(f"\nR² Score: {r2:.5f}")
print(f"MSE: {mse:.5f}")
print(f"RMSE: {rmse:.5f}")

Number of devices in original test set: 10000
Using first 10000 devices from the test set
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Time taken to predict 10000 devices: 0.624 seconds
Average predictions per second: 16020.24
Average time per prediction: 0.062 milliseconds

Prediction Error Statistics (Corrected):
Mean Absolute Percentage Error (MAPE): 0.93%
Median Absolute Percentage Error: 0.49%
Minimum Percentage Error: 0.00%
Maximum Percentage Error: 54.66%

R² Score: 0.99834
MSE: 0.00011
RMSE: 0.01060
